This code functions to characterize mRNAs in edited sequences based on a given set of reference sequences.  This code relies heavily on the Levenshtein software package.  Information on this package and download instructions can be found here:

https://pypi.org/project/python-Levenshtein/

In [1]:
import time
from Levenshtein import distance
t0=time.time()

These cells are the functions required for this code to run.

In [2]:
def extractSequences(filename):
    F=open(filename)
    content=F.readlines()
    F.close()
    sequences=list() #dummy list to hold raw sequence information
    for lines in range(len(content)):
        #find the start of the sequence
        if content[lines][0]==">":
            sequence=""
            sequence=sequence+content[lines]+content[lines+1]
            sequences.append(sequence)
    allSeq=list()
    #convert all raw sequences to the Sequence class in a new list
    for i in range(len(sequences)):
        seqi=Sequence(sequences[i])
        allSeq.append(seqi)
        if i%10000==0:
            print i
    return allSeq

In [3]:
class Sequence:
    def __init__(self,string):
        self.content=string
        #remove *s, us, and Ts
        self.content=self.content.replace("*","")
        self.content=self.content.replace("u","U")
        self.content=self.content.replace("t","U")
        self.content=self.content.replace("T","U")
        self.content=self.content.replace("a","A")
        self.content=self.content.replace("g","G")
        self.content=self.content.replace("c","C")

        #count the length of the name
        self.titleLength=0
        while self.content[self.titleLength]!=("\n"):
            self.titleLength=self.titleLength+1
        
        #make the title
        self.title=""
        for i in range(self.titleLength-1):
            self.title=self.title+self.content[i+1]#this excludes the ">" at the beginning of each line
        
        #extract the sequence by looking after the title
        self.sequence=self.content[self.titleLength+1]
        for i in range(self.titleLength+2,len(self.content)):
            self.sequence=self.sequence+self.content[i]
        self.sequence=self.sequence.replace("\n","")#remove linebreaks
        
        #Separate accession and abundance acc-abund
        self.accession=self.title[0:self.title.find("-")]
        self.abundance=int(self.title[self.title.find("-")+1:])
        
        #States list
        self.states=[[],[],[],[],[],[],[],[],[],[],[],[]]
        self.statesCompare=[]
        self.uned=["","","","","","","","","","","",""]
        
        #make the noU sequence and the U list
        self.uList=list()
        uCounter=0 #counts the number of Us at any given position
        seqPos=0 #is the placekeeper in the sequence
        self.noUs=""
        #initialize the uList
        if self.sequence[0]=="U":
            uCounter=1
            seqPos=1
            #count how many consecutive Us are at this position
            while self.sequence[uCounter]=="U":
                uCounter=uCounter+1
                seqPos=seqPos+1
            self.uList.append(uCounter)
        else:
            self.uList.append(0)
        #add each non-U nucleotide to the noUs string and count how many Us come after each nucleotide
        while seqPos<len(self.sequence):
            self.noUs=self.noUs+self.sequence[seqPos]
            seqPos=seqPos+1
            uCounter=0
            if seqPos<len(self.sequence):
                if self.sequence[seqPos]=="U":
                    uCounter=1
                    seqPos=seqPos+1
                    while seqPos<len(self.sequence) and self.sequence[seqPos]=="U":
                        uCounter=uCounter+1
                        seqPos=seqPos+1
                    self.uList.append(uCounter)
                else:
                    self.uList.append(0)
        #for some reason, it wasn't adding the U at the end every time, so I added these two lines
        if self.sequence[seqPos-1]!="U":
            self.uList.append(0)

In [4]:
def testastate(statename,stateseq,seqgroup):
    statenamemut=statename+"M"
    halfpoint=len(stateseq)/2
    for i in range(len(seqgroup)):
        if seqgroup[i].sequence.find(stateseq)>-1:
            seqgroup[i].states.append(statename)
        elif seqgroup[i].sequence.find(stateseq[0:halfpoint])>-1:
            if seqCompare(seqgroup[i].sequence,stateseq)==1:
                seqgroup[i].states.append(statenamemut)
        elif seqgroup[i].sequence.find(stateseq[halfpoint:])>-1:
            if seqCompare(seqgroup[i].sequence,stateseq)==1:
                seqgroup[i].states.append(statenamemut)
        if i%10000==0:
            print i
    return seqgroup

In [5]:
def testastateSpecExcl(statename,stateseq,seqgroup,listnum):
    statenamemut=statename+"M"
    halfpoint=len(stateseq)/2
    for i in range(len(seqgroup)):
        if len(seqgroup[i].states[listnum])==0:
            if seqgroup[i].sequence.find(stateseq)>-1:
                seqgroup[i].states[listnum].append(statename)
            elif seqgroup[i].sequence.find(stateseq[0:halfpoint])>-1:
                if seqCompare(seqgroup[i].sequence,stateseq)==1:
                    seqgroup[i].states[listnum].append(statenamemut)
            elif seqgroup[i].sequence.find(stateseq[halfpoint:])>-1:
                if seqCompare(seqgroup[i].sequence,stateseq)==1:
                    seqgroup[i].states[listnum].append(statenamemut)
        if i%10000==0:
            print i
    return seqgroup

In [6]:
def testastateSpec(statename,stateseq,seqgroup,listnum):
    statenamemut=statename+"M"
    halfpoint=len(stateseq)/2
    for i in range(len(seqgroup)):
        if seqgroup[i].sequence.find(stateseq)>-1:
            seqgroup[i].states[listnum].append(statename)
        elif seqgroup[i].sequence.find(stateseq[0:halfpoint])>-1:
            if seqCompare(seqgroup[i].sequence,stateseq)==1:
                seqgroup[i].states[listnum].append(statenamemut)
        elif seqgroup[i].sequence.find(stateseq[halfpoint:])>-1:
            if seqCompare(seqgroup[i].sequence,stateseq)==1:
                seqgroup[i].states[listnum].append(statenamemut)
        if i%10000==0:
            print i
    return seqgroup

In [7]:
def testExclusivestate(statename,stateseq,seqgroup):
    statenamemut=statename+"M"
    halfpoint=len(stateseq)/2
    for i in range(len(seqgroup)):
        if len(seqgroup[i].states)==0:
            if seqgroup[i].sequence.find(stateseq)>-1:
                seqgroup[i].states.append(statename)
            elif seqgroup[i].sequence.find(stateseq[0:halfpoint])>-1:
                if seqCompare(seqgroup[i].sequence,stateseq)==1:
                    seqgroup[i].states.append(statenamemut)
            elif seqgroup[i].sequence.find(stateseq[halfpoint:])>-1:
                if seqCompare(seqgroup[i].sequence,stateseq)==1:
                    seqgroup[i].states.append(statenamemut)
        if i%10000==0:
            print i
    return seqgroup

In [8]:
def seqCompare(seq1,seq2):
    if len(seq1)>len(seq2):
        n=0
        bestScore=1000
        while n<=(len(seq1)-len(seq2)):
            currentScore=distance(seq1[n:(len(seq2)+n)],seq2)
            if currentScore<bestScore:
                bestScore=currentScore
            n=n+1
        if bestScore>1:
            n=0
            while n<=(len(seq1)-(len(seq2)+1)):
                currentScore=distance(seq1[n:(len(seq2)+n+1)],seq2)
                if currentScore<bestScore:
                    bestScore=currentScore
                n=n+1
            if bestScore>1:
                n=0
                while n<=(len(seq1)-(len(seq2)-1)):
                    currentScore=distance(seq1[n:(len(seq2)+n-1)],seq2)
                    if currentScore<bestScore:
                        bestScore=currentScore
                    n=n+1
    else:
        if len(seq2)>len(seq1):
            n=0
            bestScore=1000
            while n<=(len(seq2)-len(seq1)):
                currentScore=distance(seq2[n:(len(seq1)+n)],seq1)
                if currentScore<bestScore:
                    bestScore=currentScore
                n=n+1
            if bestScore>1:
                n=0
                while n<=(len(seq2)-(len(seq1)+1)):
                    currentScore=distance(seq2[n:(len(seq1)+n+1)],seq1)
                    if currentScore<bestScore:
                        bestScore=currentScore
                    n=n+1
                    
                if bestScore>1:
                    n=0
                    while n<=(len(seq2)-(len(seq1)-1)):
                        currentScore=distance(seq2[n:(len(seq1)+n-1)],seq1)
                        if currentScore<bestScore:
                            bestScore=currentScore
                        n=n+1
                    
        else:
            bestScore=distance(seq1,seq2)
    return bestScore

In [9]:
def manystatestogether(searchlist,statelistacc):
    totalperfect=0
    totalmm=0
    for i in range(len(datagroup)):
        if distance(datagroup[i].noUs,masterACG)<6:
            placeholder=0
            placeholdermm=0
            for j in range(len(statelistacc)):
                if searchlist[j] in datagroup[i].states[statelistacc[j]]:
                    placeholder=placeholder+1
                elif searchlist[j]+"M" in datagroup[i].states[statelistacc[j]]:
                    amibest=0
                    if datagroup[i].states[statelistacc[j]][0]!=searchlist[j]+"M":
                        amibest=-1
                    for k in range(len(datagroup[i].states[statelistacc[j]])):
                        if datagroup[i].states[statelistacc[j]][k]==searchlist[j]+"M":
                            if amibest==0:
                                amibest=1
                        if datagroup[i].states[statelistacc[j]][k][-1]!="M":
                            amibest=-1
                    if amibest==1:
                        placeholdermm=placeholdermm+1
            if placeholder==len(searchlist):
                totalperfect=totalperfect+datagroup[i].abundance
            elif (placeholder+placeholdermm)==len(searchlist):
                totalmm=totalmm+datagroup[i].abundance
    return totalperfect,totalmm

This cell opens and reads the states file.  The states file needs the sequences of each editing block in their fully edited and unedited forms.  The states can be of any length as long as all states within an editing block have the same ACG nucleotides.  To ensure full coverage of the data set, the nucleotides on the ends of the blocks should always be ACGs and should be shared with the neighboring editing blocks.  

For example:

Full sequence: "ACGUCUAGUAGUGAUCUAUCUG"

Block B: "ACGUCUAGUAG"

Block A: "GUGAUCUAUCUG"

The states file has a fasta format and each state can have any name, but the first character must be the name of the block that state is a part of, and the state name must end in -0.

Example:
">A001-0

GUGAUCUAUCUG"

Unedited states should start with the block level character and end with Uned

Example:
">AUned-0"


The state name list just a list of the names of each block.  Set the block names as one character.  The output of this cell gives a list of all states that fall under that block as well as the individual sequences of the states

In [10]:
statesfile="teststates.fa"
statesgroup=extractSequences(statesfile)
statenamelist=["A","B","C"]
statelistlist=[]
for j in range(len(statenamelist)):
    statelistlist.append([])
    for i in range(len(statesgroup)):
        if statesgroup[i].accession[0]==statenamelist[j]:
            statelistlist[j].append(i)
    print statenamelist[j]+"list="+str(statelistlist[j])

for i in range(len(statenamelist)):
    for j in range(len(statelistlist[i])):
        print statesgroup[statelistlist[i][j]].accession+","+statesgroup[statelistlist[i][j]].sequence
        


0
Alist=[0, 3]
Blist=[1, 4]
Clist=[2, 5]
AUned,ACGAGCAGUCGCAUGGUUUUUUGG
A001,ACGAGCAGUUUCGUCAUUUUGUUUGUUGUG
BUned,GGCACUAAUUUUUAGCGUUUACGCA
B001,GUUUUGUUCUUUUACUUUUAUUUAUUUUAGUUUUCGUUAUUUUCUUUGUUUCUA
CUned,AAGUUAGAGGUUUCGCGUUUUUAGGA
C001,AUAUUUUGUUUUAUUUUGUUUAGGUUUUCUUUGCUUUUGUUUUUAGGA


This cell extracts the sequences from the data files. Data files must be in fasta format with names formatted as follows

">AccessionNumber-Reads"

Example:
">239402385-12"

Output of this cell shows the progress of the extraction and time elapsed.

In [11]:
t0=time.time()

statesfile="teststates.fa"
datafile="testmRNAs.fa"

print "Extracting Sequences"
statesgroup=extractSequences(statesfile)
datagroup=extractSequences(datafile)
t1=time.time()
print t1-t0
print "Sequences extracted"

Extracting Sequences
0
0
0.00399994850159
Sequences extracted


This cell tests for all possible states.  For each state it looks for perfect matches and matches with a single mismatch.  

The output of this cell shows the progress of the search.

In [12]:
t0=time.time()

for m in range(len(statelistlist)):
    print "Testing "+str(len(statelistlist[m]))+" states for population "+statenamelist[m]+"."
    for i in statelistlist[m]:
        mindistance=100
        for j in range(statelistlist[m].index(i)):
            testdistance=distance(statesgroup[i].sequence,statesgroup[statelistlist[m][j]].sequence)
            if testdistance>0:
                if testdistance<mindistance:
                    mindistance=testdistance
        print "State "+statesgroup[i].accession
        if mindistance>2:
            datagroup=testastateSpecExcl(statesgroup[i].accession,statesgroup[i].sequence,datagroup,m)
        else:
            datagroup=testastateSpec(statesgroup[i].accession,statesgroup[i].sequence,datagroup,m)
        tx=time.time()
        print tx-t0
    for j in range(len(datagroup)):
        if len(datagroup[j].states[m])>0:
            unedited="yes"
            for k in range(len(datagroup[j].states[m])):
                if datagroup[j].states[m][k].find("Uned")==-1:
                    unedited="no"
            if unedited=="yes":
                datagroup[j].uned[m]="unedited"
            else:
                datagroup[j].uned[m]="edited"
    
t2=time.time()
print t2-t0
  


Testing 2 states for population A.
State AUned
0
0.00400018692017
State A001
0
0.00500011444092
Testing 2 states for population B.
State BUned
0
0.00800013542175
State B001
0
0.00800013542175
Testing 2 states for population C.
State CUned
0
0.0090000629425
State C001
0
0.0170001983643
0.018000125885


This cell prints out all counts filtered and unfiltered once the previous cell is completed.  States with an "M" at the end of the name show the counts of that state for sequences that differed from the state by a signle mismatch.  This cell requires the input of the ACG sequence of the mRNA.  This allows the program to filter out sequences that have mismatches to the ACG sequence greater than or equal to the cutoff.  


In [13]:
masterACG="ACGAGCAGCGCAGGGGGCACAAAGCGACGCAAGAGAGGCGCGAGGA"
cutoff=6

for k in range(len(statelistlist)):
    statecombolist=[]
    statecomboabund=[]
    statecomboabundStringent=[]
    for i in range(len(datagroup)):
        if datagroup[i].states[k] not in statecombolist:
            statecombolist.append(datagroup[i].states[k])
            statecomboabund.append(0)
            statecomboabundStringent.append(0)
        position=statecombolist.index(datagroup[i].states[k])
        statecomboabund[position]=statecomboabund[position]+datagroup[i].abundance
        if distance(datagroup[i].noUs,masterACG)<cutoff:
            statecomboabundStringent[position]=statecomboabundStringent[position]+datagroup[i].abundance
    for j in range(len(statecombolist)):
        if len(statecombolist[j])==0:
            print str(statenamelist[k])+"Unaccounted"+"#"+str(statecomboabund[j])+"#"+str(statecomboabundStringent[j])
        elif len(statecombolist[j])==1:
            print str(statecombolist[j][0])+"#"+str(statecomboabund[j])+"#"+str(statecomboabundStringent[j])
        elif len(statecombolist[j])>1:
            keepingMain=0
            firstgood=0
            for q in range(len(statecombolist[j])):
                if firstgood==0:
                    if statecombolist[j][q][-1]!="M":
                        keepingMain=q
                        firstgood=1
            statecombolist[j]=[statecombolist[j][keepingMain]]
            print str(statecombolist[j][0])+"#"+str(statecomboabund[j])+"#"+str(statecomboabundStringent[j])

AUned#175#175
A001#170#170
AUnaccounted#65#50
BUned#103#103
B001#220#220
BUnaccounted#85#70
BUnedM#2#2
CUned#117#102
C001#210#210
C001M#80#80
CUnaccounted#3#3


This cell counts all the transcripts that are fully unedited and outputs the number of fully unedited reads before and after the filter. It does include sequences that have a single mismatch in the unedited state as well.  

In [14]:
countuned=0
countunedfil=0
for i in range(len(datagroup)):
    unedited=0
    for j in range(len(statenamelist)):
        target=statenamelist[j]+"Uned"
        targetM=target+"M"
        if target in datagroup[i].states[j]:
            unedited=unedited+1
        elif targetM in datagroup[i].states[j]:
            if len(datagroup[i].states[j])==1:
                unedited=unedited+1
            elif datagroup[i].states[j][0]==targetM:
                unedited=unedited+1
    if unedited==len(statenamelist):
        countuned=countuned+datagroup[i].abundance
        if distance(datagroup[i].noUs,masterACG)<6:
            countunedfil=countunedfil+datagroup[i].abundance

        

        
print countuned
print countunedfil

102
102


This cell prints out which states go together between editing blocks.  teststub requires the input of two block editing levels.

The output of this cell gives the filtered number of reads that have both listed states.  

In [15]:
teststub=["A","B"]
compare2=list()
for q in range(len(teststub)):
    compare2.append(statenamelist.index(teststub[q]))

gofor=len(compare2)

for i in range(gofor):
    compare1=compare2.pop(0)
    musthavestub=teststub.pop(0)
    for w in range(len(compare2)):
        #print "i'm working"

        for i in range(len(datagroup)):
            #if i%10000==0:
                #print "going through the data"
            datagroup[i].statesCompare=[]
            datagroup[i].statesCompare=datagroup[i].states[compare1]+datagroup[i].states[compare2[w]]


        statecombolist=[]
        statecomboabund=[]

        for i in range(len(datagroup)):
            #if i%10000==0:
                #print "something else is happening"
            if distance(datagroup[i].noUs,masterACG)<6:
                if datagroup[i].statesCompare not in statecombolist:
                    statecombolist.append(datagroup[i].statesCompare)
                    statecomboabund.append(0)
                position=statecombolist.index(datagroup[i].statesCompare)
                statecomboabund[position]=statecomboabund[position]+datagroup[i].abundance


        for j in range(len(statecombolist)):
            if len(statecombolist[j])>1:
                if musthavestub == statecombolist[j][0][0]:
                    if teststub[w] == statecombolist[j][-1][0]:
                #if musthavestub == statecombolist[j][0][0:2]:
                    #if teststub[w] == statecombolist[j][-1][0:2]:
                        if len(statecombolist[j])>2:
                            keepingMain=0
                            keepingTest=0
                            foundTestYet="No"
                            for q in range(len(statecombolist[j])):
                                if foundTestYet=="No":
                                    if statecombolist[j][q][0]==musthavestub:
                                    #if statecombolist[j][q][0:2]==musthavestub:
                                        if statecombolist[j][q][-1]!="M":
                                            keepingMain=q
                                    else:
                                        foundTestYet="Yes"
                                        keepingTest=q
                                if foundTestYet=="Yes":
                                    if statecombolist[j][keepingTest][-1]=="M":
                                        if statecombolist[j][q][-1]!="M":
                                            keepingTest=q
                            statecombolist[j]=[statecombolist[j][keepingMain],statecombolist[j][keepingTest]]

                        #print str(statecombolist[j])+" - "+str(statecomboabund[j])

        newcombolist=[]
        newcomboabund=[]
        for k in range(len(statecombolist)):
            if statecombolist[k] not in newcombolist:
                newcombolist.append(statecombolist[k])
                newcomboabund.append(0)
            position=newcombolist.index(statecombolist[k])
            newcomboabund[position]=newcomboabund[position]+statecomboabund[k]

        for l in range(len(newcombolist)):
            if len(newcombolist[l])>1:
                if musthavestub == newcombolist[l][0][0]:
                    if teststub[w] == newcombolist[l][-1][0]:
                #if musthavestub == newcombolist[l][0][0:2]:
                    #if teststub[w] == newcombolist[l][-1][0:2]:
                        print str(newcombolist[l])+" - "+str(newcomboabund[l])


['AUned', 'BUned'] - 103
['A001', 'B001'] - 170
['AUned', 'BUnedM'] - 2


This cell prints out the number of sequences containing a combination of upwards of two states. First number in the output is perfect, second number has at least one state with a mismatch.


searchlist requires the specific name of the states being tested.
statelistacc requires the position of the block level each state is from in the statenamelist

In [16]:
searchlist=['A001','B001','C001']

compare3=list()
for q in range(len(searchlist)):
    compare3.append(statenamelist.index(searchlist[q][0]))

manystatestogether(searchlist,compare3)

(90, 80)

This cell pulls out all mRNAs that don't have a match to any of the references sequences for each editing block.  Only sequences above the input cutoff are pulled.  

The output of this cell shows the accession number of the sequence, the number of reads of the sequence, and the sequence itself.  

In [17]:
cutoff=1
for i in range(len(statelistlist)):
    print "Block "+statenamelist[i]
    for j in range(len(datagroup)):
        check=distance(datagroup[j].noUs,masterACG)
        if check<6:
            if len(datagroup[j].states[i])==0:
                if datagroup[j].abundance>cutoff:
                    print datagroup[j].accession+","+str(datagroup[j].abundance)+","+datagroup[j].sequence

Block A
2827475,50,ACGAGCAGUCUUGCAUGGUUGGUUUUGUUCUUUUACUUUUAUUUAUUUUAGUUUUCGUUAUUUUCUUUGUUUCUAUAUUUUGUUUUAUUUUGUUUAGGUUUUCUUUGCUUUUGUUUUUAGGA
Block B
29349324,70,ACGAGCAGUCGCAUGGUUUUUUGGGCACUAAUUUAGCGACGUUUCUAUAUUUUGUUUUAUUUUGUUUAGGUUUUCUUUGCUUUUGUUUUUAGGA
Block C
5043,3,ACGAGCAGUCGCAUGGUUUUUUGGGCACUAAUUUUUAGCGUUUACGCAAGUAUUUUUUUGUAGGCGCUUUUGUUUUUAGGA


This cell looks for one specific sequence in every mRNA regardless of editing block.  

The outputs of this cell are all sequences containing the testseq above a set cutoff and the number of reads with a perfect match and the number of reads with a single mismatch to the testseq.

In [18]:
testseq="GUUCUUUUAC"
cutoff=5
testseq=testseq.replace("u","U")
print testseq
halfpoint=int(len(testseq)/2)
tophalf=testseq[0:halfpoint]
bottomhalf=testseq[halfpoint:]
tsreads=0
tsmmreads=0
t0=time.time()
for i in range(len(datagroup)):
    go=0
    if distance(datagroup[i].noUs,masterACG)<6:
        go=go+1
    if datagroup[i].sequence.find(tophalf)!=-1:
        go=go+1
    elif datagroup[i].sequence.find(bottomhalf)!=-1:
        go=go+1
    if go==2:
        newtest=seqCompare(testseq,datagroup[i].sequence)
        if newtest==0:
            tsreads=tsreads+datagroup[i].abundance
            if datagroup[i].abundance>cutoff:
                print datagroup[i].accession+"-"+str(datagroup[i].abundance)+"-"+datagroup[i].sequence
        elif newtest==1:
            tsmmreads=tsmmreads+datagroup[i].abundance
    #if i%10000==0:
        #t1=time.time()
        #print str(i)+"-"+str(tsreads)+"-"+str(tsmmreads)+"-"+str(t1-t0)
print tsreads
print tsmmreads


GUUCUUUUAC
8234998234-90-ACGAGCAGUUUCGUCAUUUUGUUUGUUGUGUUUUGUUCUUUUACUUUUAUUUAUUUUAGUUUUCGUUAUUUUCUUUGUUUCUAUAUUUUGUUUUAUUUUGUUUAGGUUUUCUUUGCUUUUGUUUUUAGGA
2342934-80-ACGAGCAGUUUCGUCAUUUUGUUUGUUGUGUUUUGUUCUUUUACUUUUAUUUAUUUUAGUUUUCGUUAUUUUCUUUGUUUCUAUAUUUUGUUUAUUUUGUUUAGGUUUUCUUUGCUUUUGUUUUUAGGA
2827475-50-ACGAGCAGUCUUGCAUGGUUGGUUUUGUUCUUUUACUUUUAUUUAUUUUAGUUUUCGUUAUUUUCUUUGUUUCUAUAUUUUGUUUUAUUUUGUUUAGGUUUUCUUUGCUUUUGUUUUUAGGA
220
0
